In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tx = open("text.txt",'r').read()

In [3]:
vocab = tx.split(" ")
print("# of total words: ", len(vocab))
vocab_set = set(vocab)
print("# of unique words: ", len(vocab_set))

# of total words:  12391
# of unique words:  3851


## Hash table

In [4]:
class Node:
    def __init__(self, key, prev=None, nxt=None, val=None):
        self.key = key
        self.prev = prev
        self.next = nxt
        self.val = val
        

class DoublyLinkedList:
    """
    structure:
    dummy -> new node -> old node
    """
    def __init__(self, head:Node):
        self.dummy = Node(key=None, prev=None, nxt=head)
        self.dummy.next.prev = self.dummy
        self.size = 1
    
    def __len__(self):
        return self.size
        
    # def __init__(self):
    #     self.dummy = Node(key=None, prev=None, nxt=None)
    #     self.size = 0
    
    # def __init__(self, key:int):
    #     self.head = Node(key)
    #     self.dummy = Node(key=None, prev=None, nxt=self.head)
    #     self.head.prev = self.dummy
    #     self.size = 1
    
    def delete(self, node):
        node.prev.next = node.next
        if node.next:
            node.next.prev = node.prev
        self.size -= 1
        if self.size == 0:
            print("empty linked list")
        
    def search(self, key):
        x = self.dummy.next
        while x and x.key != key:
            x = x.next
        return x
    
    def insert(self, x):
        x.next = self.dummy.next
        self.dummy.next.prev = x
        self.dummy.next = x
        x.prev = self.dummy
        self.size += 1
    
    def print_dll(self):
        curr = self.dummy
        while curr:
            print(curr.key, end="->")
            curr = curr.next
        print("")
        
    def delete_by_key(self, key):
        node = self.search(key)
        self.delete(node)
    
    

In [5]:
h = Node(0)
dll = DoublyLinkedList(h)
dll.insert(Node(1))
dll.insert(Node(2))
dll.insert(Node(3))
dll.insert(Node(4))
dll.print_dll()
print("length: ",dll.size)

None->4->3->2->1->0->
length:  5


In [6]:
dll.delete_by_key(2)
dll.print_dll()
print("length: ",dll.size)

None->4->3->1->0->
length:  4


In [7]:
curr = dll.dummy.next
while curr:
    dll.delete(curr)
    #dll.print_dll()
    curr = curr.next

empty linked list


In [8]:
class Hashtable:
    def __init__(self, hmax):
        self.hmax = hmax
        self.table = [None] * hmax
        self.keys = set()
        
    def get_keys(self):
        return list(self.keys)
    
    def __getitem__(self, key):
        index = self._hash(key)
        if self.table[index] is None:
            return None
        else:
            return self.table[index].search(key).val
        
    def __setitem__(self, key, val):
        index = self._hash(key)
        if self.table[index] is None:
            self.table[index] = DoublyLinkedList(Node(key=key, val=val))
        else:
            self.table[index].search(key).val = val
        
    
    def chash(self, key): # customized hash function
        return hash(key)
    
    def djb2_hash(self, key):
        hash_value = 5381
        for char in key:
            hash_value = ((hash_value << 5) + hash_value) + ord(char)
        return hash_value
    
    def _hash(self, key): # hash function wrapper
        return self.chash(key) % self.hmax
    
    def insert(self, key, val):
        self.keys.add(key)
        index = self._hash(key)
        if self.table[index] is None:
            self.table[index] = DoublyLinkedList(Node(key=key, val=val))
        else:
            self.table[index].insert(Node(key=key, val=val))
    
    # bug here
    def delete(self, key):
        self.keys.remove(key)
        index = self._hash(key)
        if self.table[index] is None:
            raise KeyError()
        else:
            res = self.table[index].search(key)
            if res:
                return res
            else:
                raise KeyError()
    
    def increase(self, key):
        self.keys.add(key)
        index = self._hash(key)
        if self.table[index] is None:
            self.table[index] = DoublyLinkedList(Node(key=key, val=1))
        else:
            res = self.table[index].search(key)
            if res:
                res.val += 1
            else:
                self.table[index].insert(Node(key=key, val=1))
    
    def get(self, key):
        index = self._hash(key)
        if self.table[index] is None:
            return None
        else:
            return self.table[index].search(key).val
        
    def count_collisions(self):
        count = []
        for dll in self.table:
            c = 0
            if dll:
                curr = dll.dummy.next
                while curr:
                    c += curr.val
                    curr = curr.next
                count.append(c)
            else:
                count.append(0)
        return count
        
            
        
    

In [9]:
# add the input text to a hashtable, character by character

hmax = 30
n = len(vocab)
ht = Hashtable(hmax)
for word in vocab:
    ht.increase(word)
n_colli = ht.count_collisions()
n/hmax, np.mean(n_colli), np.std(n_colli)

(413.03333333333336, 413.03333333333336, 223.5827189704567)

In [ ]:
ht2 = Hashtable(hmax)
for word in vocab:
    ht[word] = ht.get(word) + 1

In [10]:
hmax = 300
ht = Hashtable(hmax)
for word in vocab:
    ht.increase(word)
n_colli = ht.count_collisions()
n/hmax, np.mean(n_colli), np.std(n_colli)

(41.303333333333335, 41.303333333333335, 72.07355956860248)

In [11]:
hmax = 1000
ht = Hashtable(hmax)
for word in vocab:
    ht.increase(word)
n_colli = ht.count_collisions()
n/hmax, np.mean(n_colli), np.std(n_colli)

(12.391, 12.391, 39.902106698769686)